# Thread Pool:
- A thread pool is a collection of worker threads that are used to perform tasks concurrently. The primary goal of a thread pool is to avoid the overhead of creating and destroying threads for each task, by reusing threads for multiple tasks. Python provides the ThreadPoolExecutor class from the concurrent.futures module to manage and use thread pools effectively.

### Why Use a Thread Pool?
- Efficiency: Creating a new thread for every task can be resource-intensive. A thread pool allows reusing threads to handle multiple tasks, reducing overhead.
- Simplification: Managing a thread pool abstracts the complexities of thread creation, execution, and destruction, allowing the programmer to focus on the task logic.


### ThreadPoolExecutor Methods:
#### i.submit(func, *args, **kwargs)
- The submit() method schedules a callable (function) to be executed asynchronously in the thread pool. It returns a Future object, which allows you to track the execution of the callable, including checking its status and retrieving the result.

- Usage: This is useful when you need to run a function asynchronously and don't need to block the main thread while waiting for the result.

Parameters:
- func: The function to execute.
- args: The positional arguments to pass to the function.
- kwargs: The keyword arguments to pass to the function.

Returns:
- A Future object that represents the execution of the callable.

In [3]:
from concurrent.futures import ThreadPoolExecutor

def square(x):
    return x ** 2

with ThreadPoolExecutor(max_workers=3) as executor:
    future1 = executor.submit(square, 2)
    future2 = executor.submit(square, 3)
    print(f"Result of future1: {future1.result()}")
    print(f"Result of future2: {future2.result()}")


Result of future1: 4
Result of future2: 9


### ii. map(func, iterable, timeout=None, chunksize=1)
 The map() method applies a function (func) to every item of the iterable (iterable) concurrently. The behavior is similar to the built-in map() function but it distributes the work to multiple threads. It returns the results in the same order as the input iterable.

- Usage: Use map() when you need to apply a function to an iterable (like a list) and get the results as a list. It blocks until all results are returned.

Parameters:
- func: The function to apply to each item in the iterable.
- iterable: The input iterable (list, tuple, etc.).
- timeout: (Optional) The maximum number of seconds to wait for the results.
- chunksize: (Optional) The number of items each thread will process at once.

Returns: 
- An iterable of results in the same order as the input iterable.

In [4]:
from concurrent.futures import ThreadPoolExecutor

def square(x):
    return x ** 2

with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(square, [1, 2, 3, 4, 5]))
    print(results)  # Output: [1, 4, 9, 16, 25]


[1, 4, 9, 16, 25]


### iii. shutdown(wait=True)
- This method is used to clean up the pool. By calling shutdown(), you prevent any more tasks from being added to the pool, and you can optionally wait for the running threads to finish before proceeding.

- Usage: Call this method when you are done with the ThreadPoolExecutor to release its resources. If wait is set to True, it will block until all tasks are completed.

When wait is:
- True: Blocks the main program until all threads finish their tasks.
- False: Immediately shuts down the pool but lets running threads continue in the background.

Returns:
- None

In [12]:
from concurrent.futures import ThreadPoolExecutor

def square(x):
    return x ** 2

with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(square, [1, 2, 3, 4, 5]))
    print(results)  # Output: [1, 4, 9, 16, 25]
    executor.shutdown(wait=True)  # Wait for all threads to finish


[1, 4, 9, 16, 25]


### Problem with threading.Thread:
- Number of threads: For each task, a new thread is created, which can be inefficient when there are many tasks.
- Non-reusable threads: Once a thread completes its task, it cannot be reused. It must be recreated for the next task which is resource and time inefficient.

In [7]:
import threading
import time
import os

# Function to simulate task
def task(x):
    print(f"Task {x} starting (Thread ID: {threading.get_ident()})")
    time.sleep(2)  # Simulating a time-consuming task
    print(f"Task {x} finished (Thread ID: {threading.get_ident()})")

# Create threads for each task
threads = []
for i in range(10):
    thread = threading.Thread(target=task, args=(i,))
    threads.append(thread)
    thread.start()  # Start the thread

# Wait for all threads to complete
for thread in threads:
    thread.join()

print("All tasks completed with threading.Thread.")


Task 0 starting (Thread ID: 13652)
Task 1 starting (Thread ID: 11908)
Task 2 starting (Thread ID: 22716)
Task 3 starting (Thread ID: 3112)
Task 4 starting (Thread ID: 18736)
Task 5 starting (Thread ID: 8376)
Task 6 starting (Thread ID: 24476)
Task 7 starting (Thread ID: 1480)
Task 8 starting (Thread ID: 22308)
Task 9 starting (Thread ID: 9780)
Task 0 finished (Thread ID: 13652)
Task 1 finished (Thread ID: 11908)
Task 2 finished (Thread ID: 22716)
Task 3 finished (Thread ID: 3112)
Task 4 finished (Thread ID: 18736)
Task 5 finished (Thread ID: 8376)
Task 6 finished (Thread ID: 24476)
Task 7 finished (Thread ID: 1480)
Task 8 finished (Thread ID: 22308)
Task 9 finished (Thread ID: 9780)
All tasks completed with threading.Thread.


### Solution with ThreadPoolExecutor:
- Thread reuse: Threads are reused from the pool, reducing the overhead.
- Efficient thread allocation: The pool controls the maximum number of threads running simultaneously.

In [8]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading

# Function to simulate task
def task(x):
    print(f"Task {x} starting (Thread ID: {threading.get_ident()})")
    time.sleep(2)  # Simulating a time-consuming task
    print(f"Task {x} finished (Thread ID: {threading.get_ident()})")

# Create a thread pool with a maximum of 3 threads
with ThreadPoolExecutor(max_workers=3) as executor:
    # Submit tasks to the pool
    for i in range(10):
        executor.submit(task, i)

print("All tasks completed with ThreadPoolExecutor.")


Task 0 starting (Thread ID: 3692)
Task 1 starting (Thread ID: 9408)
Task 2 starting (Thread ID: 3708)
Task 0 finished (Thread ID: 3692)Task 1 finished (Thread ID: 9408)
Task 3 starting (Thread ID: 9408)
Task 2 finished (Thread ID: 3708)
Task 4 starting (Thread ID: 3708)

Task 5 starting (Thread ID: 3692)
Task 4 finished (Thread ID: 3708)Task 3 finished (Thread ID: 9408)
Task 6 starting (Thread ID: 9408)
Task 5 finished (Thread ID: 3692)
Task 7 starting (Thread ID: 3692)

Task 8 starting (Thread ID: 3708)
Task 6 finished (Thread ID: 9408)Task 8 finished (Thread ID: 3708)
Task 9 starting (Thread ID: 3708)
Task 7 finished (Thread ID: 3692)

Task 9 finished (Thread ID: 3708)
All tasks completed with ThreadPoolExecutor.
